# LINHA <> - Caderno de Operação

## Caderno de Inicialização e Calibração inicial da Linha de Luz

### Técnico responsável:

Texto inicial










## Notebook Initialization

The next cell loads the <b><i>util.ipynb</i></b> notebook, which provides functions and classes to connect the IJupyter and EPICS interfaces.

<sup><sup>Any bug or unexpected behavior from any of these functions and classes, please contact SOL.</sup></sup>

In [1]:
import io, os, sys, types
from IPython import get_ipython
import nbformat
from IPython.core.interactiveshell import InteractiveShell


def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path


class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = nbformat.read(f, nbformat.NO_CONVERT)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
            for cell in nb.cells:
                if cell.cell_type == 'code':
                    # transform the input to executable Python
                    code = self.shell.input_transformer_manager.transform_cell(cell.source)
                    # run the code in themodule
                    exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod


class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

sys.meta_path.append(NotebookFinder())

In [2]:
from util import *

importing Jupyter notebook from util.ipynb


## Configuration Cell

Check the boxes to set some notebook's configuration

In [3]:
checkbox_logprint_in_cell = widgets.Checkbox(
    value=False,
    description="Print log in Notebook's cells",
    disabled=False,
    style={'description_width': 'initial'},
)

config = {"log_cell": checkbox_logprint_in_cell}

output = widgets.Output()

@output.capture()
def change_checkbox_logprint_in_cell(change):
    logprint("Changed Logprint Checkbox to: " + str(change.new))
    
checkbox_logprint_in_cell.observe(change_checkbox_logprint_in_cell, 'value')

box = widgets.VBox([checkbox_logprint_in_cell, output])

display(box)



## Motor Calibration

### Write in the Text Box, separated by spaces, every motor name that should be initialized.

###### PS: Do not use any other character separation other than spaces!

In [4]:
config

{'log_cell': Checkbox(value=False, description="Print log in Notebook's cells", style=DescriptionStyle(description_width='initial'))}

In [5]:
start_motor_button_box = StartMotorsButton(config)
start_motor_button_box.display_start_button()

Output()

### Write in the "Number Text Box" the absolute target value desired for the motor 

In [6]:
start_motor_button_box.display_motors_targ_buttons()

### Write in the Text Box, separated by spaces, every motor PV name that should be monitored.

###### PS: Do not use any other character separation other than spaces!

In [7]:
motor_monitor = MotorsMonitor(config)
motor_monitor.display_monitor_motors()

Output()

In [8]:
# from py4syn.utils.scan import scan
# scan(cnew_motor, 10, 5, 10, 0.1)

In [9]:
pv_monitor = PVMonitor(config)
pv_monitor.display_monitor_pvs()

Output()

In [10]:
scan_but = ScanButton(config)
scan_but.display_scan()

Output()

## Plotting data

### Plotly

Plotly is a great library for plotting data in a Jupyter Notebook.

Informations and examples of how to use this library can be found in https://plot.ly/python/


#### Simple scatter plot example:

In [17]:
# Enable offline plots from plotly
py.init_notebook_mode(connected=True)

# Read sample csv file with pandas library (CSV example obatained from https://catalog.data.gov/dataset/age-adjusted-death-rates-for-the-top-10-leading-causes-of-death-united-states-2013/resource/0e603f1d-31bf-4809-8f10-a994b305b379)
df = pd.read_csv('sample-data.csv')

# Let's plot the number of total deaths for every USA state
data = df.groupby('State').sum().reset_index()

# Looking at our data, there is a United States row that includes the sum of all states, we should remove this row
for i in range(len(data.index) - 1):
    if data.iloc[i]['State'] == 'United States':
        data = data.drop(i, axis=0)

# Create a Scatter graph
trace1 = go.Scatter(
                    x=data['State'], y=data['Deaths'], # Data
                    mode='lines+markers', name='logx' # Additional options
                   )

# Create a Layout for the graph
layout = go.Layout(title='Number of deaths for every USA State',
                   plot_bgcolor='rgb(230, 230, 230)')

# Create a figure to be plotted
fig = go.FigureWidget(data=[trace1], layout=layout)

# Plot figure in the notebook
df.columns

Index(['Year', '113 Cause Name', 'Cause Name', 'State', 'Deaths',
       'Age-adjusted Death Rate'],
      dtype='object')

In [37]:
x = np.arange(1000) * 1/2
y = x * 2 + 9

s = pd.DataFrame({'x':x, 'y':y})
s

,x,y
0,0.0,9.0
1,0.5,10.0
2,1.0,11.0
3,1.5,12.0
4,2.0,13.0
5,2.5,14.0
6,3.0,15.0
7,3.5,16.0
8,4.0,17.0
9,4.5,18.0


In [18]:
df = pd.read_csv('scans/test_0012', sep=' ', comment='#', header=None)

labels = []
with open('scans/test_0012') as file:
    for i, line in enumerate(file):
        if i == 7:
            labels = line.split(' ')[1:]

labels = list(filter(lambda x: x != '', labels))

filtered_label = []

for item in labels:
    filtered_label.append(item.rstrip('\n'))
    print(item)

df.columns = pd.Index(filtered_label, dtype='object')
df

solm3
solm2
ringcurrent
virtual2



,solm3,solm2,ringcurrent,virtual2
0,1.0,4.0,223.415,51.0
1,1.1,4.0,223.415,29.0
2,1.2,4.0,223.415,21.0
3,1.3,4.0,223.415,89.0
4,1.4,4.0,223.381,7.0
5,1.5,4.0,223.381,5.0
6,1.6,4.0,223.381,25.0
7,1.7,4.0,223.381,12.0
8,1.8,4.0,223.381,34.0
9,1.9,4.0,223.346,67.0


In [ ]:
trace1 = go.Scatter(
                    x=df['gap2'], y=df['ringcurrent'], # Data
                    mode='lines+markers', name='logx' # Additional options
                   )

# Create a Layout for the graph
layout = go.Layout(title='Number of deaths for every USA State',
                   plot_bgcolor='rgb(230, 230, 230)')

# Create a figure to be plotted
fig = go.FigureWidget(data=[trace1], layout=layout)

In [ ]:
fig

In [ ]:
!gedit